In [4]:
from __future__ import annotations
import typing
from datetime import date, timedelta, datetime
import numpy as np
import pandas as pd

def get_period(key: str):
    map = {
        '1-Week': 1/52,
        '1-Month': 1/12,
        '2-Month': 2/12,
        '3-Month': 3/12,
        '6-Month': 1/2,
        '1-Year': 1,
        '2-Year': 2,
    }
    return map[key]

def populate_bond_table(bond_price, today, maturity_date):
    bond_table = pd.DataFrame(index=pd.date_range(today, maturity_date), columns=['price'])
    X = [get_period(col) for col in bond_price.columns]
    Y = bond_price.loc[today].to_list()  
    for date in bond_table.index:
        tdelta = (date - today).days/365
        interpolated_y = np.interp(tdelta,X,Y)
        bond_table.loc[date]['price'] = interpolated_y
    return bond_table

class VasicekModel(object):
    def __init__(self, data: pd.DataFrame, params: typing.Dict):
        """
        b: long term mean level: All future trajectories of r will evolve around a mean level b in the long run.
        a: speed of reversion: A characterizes the velocity at which such trajectories will regroup around b.
        sigma: instantaneous volatility: measures instant by instant the amplitude of randomness
        """
        self.data = data
        self.a = params.get('speed of reversion') # 0
        self.b = params.get('long term mean level') # 0.107659718380514
        self.sigma = params.get('sigma') # 0.106212663278328
        self.maturity_date = params.get('maturity_date')
        self.dt = 1/252
    
    def generate_path(self, current_date: str)->pd.DataFrame:
        """
            N: the number steps in the path
        """
        N = (pd.to_datetime(self.maturity_date) - pd.to_datetime(current_date)).days + 1
        Rt = [0]*(N+1)
        prev_date = pd.to_datetime(current_date) - pd.DateOffset(days=1)
        Rt[0] = self.data.loc[prev_date]['Price']
        for i in range(1, N+1):
            Rt[i] = self.a*(self.b-Rt[i-1]) * self.dt + self.sigma * np.random.normal(0, np.sqrt(self.dt)) + Rt[i-1]
        Rt = Rt[1:]
        return pd.DataFrame(data=Rt, index=pd.date_range(current_date, self.maturity_date), columns=['Rate'])

In [5]:
path = '../../data/bond'

bond_yield = None
for file in os.listdir(path):
   df = pd.read_csv(os.path.join(path, file))[['Date','Price']]
   df.rename(columns={'Price':file.split(' ')[1]}, inplace=True)
   df['Date'] = pd.to_datetime(df['Date'],format='%m/%d/%Y')
   df = df.set_index('Date').iloc[::-1]
   if bond_yield is None:
      bond_yield = df
   else:
      bond_yield = pd.concat([bond_yield, df], axis=1)
bond_yield = bond_yield.interpolate()

In [6]:
bond_yield = bond_yield.reindex(sorted(bond_yield.columns, key=lambda x: get_period(x)), axis=1)
bond_yield

,1-Week,1-Month,2-Month,3-Month,6-Month,1-Year,2-Year
Date,,,,,,,
2023-01-01,0.750,0.820,0.880,0.870,1.060,1.55,NaN
2023-01-02,0.810,0.820,0.900,0.860,1.060,1.55,NaN
2023-01-03,0.810,0.820,0.900,0.860,1.060,1.55,1.242
2023-01-04,0.830,0.840,0.900,0.800,1.040,1.49,1.192
2023-01-05,0.830,0.850,0.900,0.870,1.080,1.49,1.220
...,...,...,...,...,...,...,...
2024-10-31,0.835,0.775,0.865,0.655,0.625,0.71,0.359
2024-11-01,0.845,0.795,0.865,0.615,0.595,0.70,0.290
2024-11-04,0.880,0.785,0.900,0.615,0.595,0.70,0.280


In [7]:
bond_price = pd.DataFrame(index=bond_yield.index)
for col in bond_yield.columns:
    bond_price[col] = bond_yield[col].apply(lambda x: np.exp(-x/100*get_period(col)))
bond_price

,1-Week,1-Month,2-Month,3-Month,6-Month,1-Year,2-Year
Date,,,,,,,
2023-01-01,0.999856,0.999317,0.998534,0.997827,0.994714,0.984620,NaN
2023-01-02,0.999844,0.999317,0.998501,0.997852,0.994714,0.984620,NaN
2023-01-03,0.999844,0.999317,0.998501,0.997852,0.994714,0.984620,0.975466
2023-01-04,0.999840,0.999300,0.998501,0.998002,0.994813,0.985210,0.976442
2023-01-05,0.999840,0.999292,0.998501,0.997827,0.994615,0.985210,0.975895
...,...,...,...,...,...,...,...
2024-10-31,0.999839,0.999354,0.998559,0.998364,0.996880,0.992925,0.992846
2024-11-01,0.999838,0.999338,0.998559,0.998464,0.997029,0.993024,0.994217
2024-11-04,0.999831,0.999346,0.998501,0.998464,0.997029,0.993024,0.994416
